In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import time
import warnings

warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
sub = pd.DataFrame(test['time'])

In [3]:
train = train[train['temperature'].notnull()]
train = train.fillna(method='bfill')
test = test.fillna(method='bfill')

In [5]:
train.columns = ['time', 'year', 'month', 'day', 'hour', 'min', 'sec',
                 'outdoorTemp', 'outdoorHum', 'outdoorAtmo',
                 'indoorHum', 'indoorAtmo', 'temperature']
test.columns = ['time', 'year', 'month', 'day', 'hour', 'min', 'sec',
                'outdoorTemp', 'outdoorHum', 'outdoorAtmo',
                'indoorHum', 'indoorAtmo']

In [6]:
data = pd.concat([train, test])

In [7]:
group_feats = []
for i in tqdm(['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo']):
    data['MDH_{}_mean'.format(i)] = data.groupby(['month', 'day', 'hour'], as_index=False)[i].transform('mean')
    data['MDH_{}_median'.format(i)] = data.groupby(['month', 'day', 'hour'], as_index=False)[i].transform('median')
    data['MDH_{}_max'.format(i)] = data.groupby(['month', 'day', 'hour'], as_index=False)[i].transform('max')
    data['MDH_{}_min'.format(i)] = data.groupby(['month', 'day', 'hour'], as_index=False)[i].transform('min')
    data['MDH_{}_std'.format(i)] = data.groupby(['month', 'day', 'hour'], as_index=False)[i].transform('std')
    data['MDH_{}_skew'.format(i)] = data.groupby(['month', 'day', 'hour'], as_index=False)[i].transform('skew')
    
    group_feats.append('MDH_{}_mean'.format(i))
    group_feats.append('MDH_{}_median'.format(i))

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:17<00:00,  3.48s/it]


In [9]:
# 基本交叉特征
for f1 in tqdm(['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo'] + group_feats):
    for f2 in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo'] + group_feats:
        if f1 != f2:
            colname = '{}_{}_ratio'.format(f1, f2)
            data[colname] = data[f1].values / data[f2].values

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 80.96it/s]


In [10]:
data.fillna(method='bfill', inplace=True)

In [12]:
# 历史信息提取
# dt 14-44
data['dt'] = data['day'].values + (data['month'].values - 3) * 31

for f in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo', 'temperature']:
    tmp_df = pd.DataFrame()
    for t in tqdm(range(15, 45)):
        tmp = data[data['dt'] < t].groupby(['hour'])[f].agg({'mean'}).reset_index()
        tmp.columns = ['hour', 'hit_{}_mean'.format(f)]
        tmp['dt'] = t
        tmp_df = tmp_df.append(tmp)

    data = data.merge(tmp_df, on=['dt', 'hour'], how='left')

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 38.29it/s]


In [13]:
data.fillna(method='bfill', inplace=True)

In [14]:
# 离散化
for f in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo']:
    data[f + '_20_bin'] = pd.cut(data[f], 20, duplicates='drop').apply(lambda x: x.left).astype(int)
    data[f + '_50_bin'] = pd.cut(data[f], 50, duplicates='drop').apply(lambda x: x.left).astype(int)
    data[f + '_100_bin'] = pd.cut(data[f], 100, duplicates='drop').apply(lambda x: x.left).astype(int)
    data[f + '_200_bin'] = pd.cut(data[f], 200, duplicates='drop').apply(lambda x: x.left).astype(int)

In [ ]:
for f1 in tqdm(['outdoorTemp_20_bin', 'outdoorHum_20_bin', 'outdoorAtmo_20_bin', 'indoorHum_20_bin', 'indoorAtmo_20_bin']):
    for f2 in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo']:
        data['{}_{}_median'.format(f1, f2)] = data.groupby([f1])[f2].transform('median')
        data['{}_{}_mean'.format(f1, f2)] = data.groupby([f1])[f2].transform('mean')
        data['{}_{}_max'.format(f1, f2)] = data.groupby([f1])[f2].transform('max')
        data['{}_{}_min'.format(f1, f2)] = data.groupby([f1])[f2].transform('min')

In [ ]:
for f1 in tqdm(['outdoorTemp_50_bin', 'outdoorHum_50_bin', 'outdoorAtmo_50_bin', 'indoorHum_50_bin', 'indoorAtmo_50_bin']):
    for f2 in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo']:
        data['{}_{}_median'.format(f1, f2)] = data.groupby([f1])[f2].transform('median')
        data['{}_{}_mean'.format(f1, f2)] = data.groupby([f1])[f2].transform('mean')
        data['{}_{}_max'.format(f1, f2)] = data.groupby([f1])[f2].transform('max')
        data['{}_{}_min'.format(f1, f2)] = data.groupby([f1])[f2].transform('min')

In [ ]:
for f1 in tqdm(['outdoorTemp_100_bin', 'outdoorHum_100_bin', 'outdoorAtmo_100_bin', 'indoorHum_100_bin', 'indoorAtmo_100_bin']):
    for f2 in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo']:
        data['{}_{}_median'.format(f1, f2)] = data.groupby([f1])[f2].transform('median')
        data['{}_{}_mean'.format(f1, f2)] = data.groupby([f1])[f2].transform('mean')
        data['{}_{}_max'.format(f1, f2)] = data.groupby([f1])[f2].transform('max')
        data['{}_{}_min'.format(f1, f2)] = data.groupby([f1])[f2].transform('min')

In [ ]:
for f1 in tqdm(['outdoorTemp_200_bin', 'outdoorHum_200_bin', 'outdoorAtmo_200_bin', 'indoorHum_200_bin', 'indoorAtmo_200_bin']):
    for f2 in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo']:
        data['{}_{}_median'.format(f1, f2)] = data.groupby([f1])[f2].transform('median')
        data['{}_{}_mean'.format(f1, f2)] = data.groupby([f1])[f2].transform('mean')
        data['{}_{}_max'.format(f1, f2)] = data.groupby([f1])[f2].transform('max')
        data['{}_{}_min'.format(f1, f2)] = data.groupby([f1])[f2].transform('min')

In [ ]:
nums = int(data.shape[0] * 0.8)

